<a href="https://colab.research.google.com/github/Lucasl3/grafos/blob/main/Atividade1/Lucas_Lemos_Cerqueira_de_Freitas_Atividade01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 01 - Planejamento de rota

In [ ]:
import numpy as np

## Aplicação básica

Um robô precisa planejar a menor rota em uma grade de ocupação. Suponha que a grade de ocupação é fornecida como uma matriz com células indicando espaço livre (1) e obstáculo (∞). No caso, para a grade de ocupação a seguir:

É definida respectiva matriz:

In [ ]:
occupancy = np.ones((10, 10))
occupancy[0:5, 0] = np.inf
occupancy[5, 0:5] = np.inf
occupancy[5:8, 5] = np.inf
occupancy[0:3, 5:8] = np.inf
occupancy[7:9, 3] = np.inf
occupancy[5:10, 8] = np.inf

n = len(occupancy)
m = len(occupancy[0])

print(occupancy)

[[inf  1.  1.  1.  1. inf inf inf  1.  1.]
 [inf  1.  1.  1.  1. inf inf inf  1.  1.]
 [inf  1.  1.  1.  1. inf inf inf  1.  1.]
 [inf  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [inf  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [inf inf inf inf inf inf  1.  1. inf  1.]
 [ 1.  1.  1.  1.  1. inf  1.  1. inf  1.]
 [ 1.  1.  1. inf  1. inf  1.  1. inf  1.]
 [ 1.  1.  1. inf  1.  1.  1.  1. inf  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1. inf  1.]]


Sabendo que o robô inicia em uma célula específica, e.g. pos = [9, 0], e deseja-se chegar em uma posição destino, e.g. pos_d = [0, 2]. O robô se move apenas na horizontal e vertical (não se move na diagonal). Espera-se de retorno uma lista com o caminho.

In [ ]:
def validMove(i, j, occupancy):
  if (i >= 0 and i < n) and (j >= 0 and j < m):
    position = occupancy[i, j]
    return not np.isinf(position)
  return False

def checkNeighbors(position, occupancy):
  movements = [(0, 1), (0, -1), (1, 0), (-1, 0)]
  neighbors = []
  
  for movement in movements:
    possible_i, possible_j = np.sum([position, movement], axis=0)
    if validMove(possible_i, possible_j, occupancy):
      possible_position = occupancy[possible_i, possible_j]
      neighbors.append((possible_i, possible_j))

  return neighbors

def createGraph(occupancy):
  graph = {}
  for (i, row) in enumerate(occupancy):
    for (j, item) in enumerate(row):
      if not np.isinf(item):
        current_position = (i, j)
        valid_neighbors = checkNeighbors(current_position, occupancy)
        graph[current_position] = valid_neighbors

  return graph

In [ ]:
import queue

def unvisitedNeighbors(U, matriz_distance, graph):
  return [neighbor for neighbor in graph[U] if matriz_distance[neighbor]["unvisited"]]

def getEdgeWeight(V, occupancy):
  i, j = V
  return occupancy[i, j]

def getPath(start, end, matriz_distance):
  path = []
  current_node = end
  distance = matriz_distance[current_node]["distance"]
  while current_node != start:
    path.append(current_node)
    current_node = matriz_distance[current_node]["previous"]
  path.append(start)
  path.reverse()

  return (path, distance)

def dijkstraAlgorithm(graph, start, end, occupancy):
  matriz_distance = {}
  priority_queue = queue.PriorityQueue()

  for V in graph:
    matriz_distance[V] = {
        "unvisited": True,
        "distance": np.inf,
        "previous": None
    }
    if V == start:
      matriz_distance[V]["distance"] = 0
      priority_queue.put((0, V))

  while not priority_queue.empty():
    distance, U = priority_queue.get()
    unvisited_neighbors = unvisitedNeighbors(U, matriz_distance, graph)
    for neighbor in unvisited_neighbors:
      temp_distance = distance + getEdgeWeight(neighbor, occupancy)
      if temp_distance < matriz_distance[neighbor]["distance"]:
        matriz_distance[neighbor]["distance"] = temp_distance
        matriz_distance[neighbor]["previous"] = U
        priority_queue.put((temp_distance, neighbor))
    
    matriz_distance[U]["unvisited"] = False

  return matriz_distance

In [ ]:
def printPath(path, distance, occupancy):
  for (i, node) in enumerate(path):
    if i != 0:
      print("_________________________________________________")
      print(f"From {previous_node} to {node} with cost {getEdgeWeight(node, occupancy)}")
    else:
      print("Starting at", node)
    previous_node = node
  
  print(f"\nTotal cost: {distance}")

In [ ]:
def robot_path(robot_pos, robot_pos_d, occupancy):
  robot_pos, robot_pos_d = tuple(robot_pos), tuple(robot_pos_d)
  graph = createGraph(occupancy)
  matriz_distance = dijkstraAlgorithm(graph, robot_pos, robot_pos_d, occupancy)
  path, distance = getPath(robot_pos, robot_pos_d, matriz_distance)
  return path, distance

In [ ]:
robot_pos_c = [9, 0] # Robot current position
robot_pos_d = [0, 2] # Robot desired position

path, distance = robot_path(robot_pos_c, robot_pos_d, occupancy)
printPath(path, distance, occupancy)

Starting at (9, 0)
_________________________________________________
From (9, 0) to (9, 1) with cost 1.0
_________________________________________________
From (9, 1) to (9, 2) with cost 1.0
_________________________________________________
From (9, 2) to (9, 3) with cost 1.0
_________________________________________________
From (9, 3) to (9, 4) with cost 1.0
_________________________________________________
From (9, 4) to (8, 4) with cost 1.0
_________________________________________________
From (8, 4) to (8, 5) with cost 1.0
_________________________________________________
From (8, 5) to (8, 6) with cost 1.0
_________________________________________________
From (8, 6) to (7, 6) with cost 1.0
_________________________________________________
From (7, 6) to (6, 6) with cost 1.0
_________________________________________________
From (6, 6) to (5, 6) with cost 1.0
_________________________________________________
From (5, 6) to (4, 6) with cost 1.0
___________________________________

## Considerando terreno

Agora vamos considerar que algumas células representam regiões com terreno de mais difícil acesso que outras (terreno íngreme, com maior atrito). Dessa vez as células da grade de ocupação terão um peso atribuído de maneira não uniforme. 

In [ ]:
occupancy = 10 * np.random.rand(10, 10)
occupancy[0:5, 0] = np.inf
occupancy[5, 0:5] = np.inf
occupancy[5:8, 5] = np.inf
occupancy[0:3, 5:8] = np.inf
occupancy[7:9, 3] = np.inf
occupancy[5:10, 8] = np.inf
n = len(occupancy)
m = len(occupancy[0])

for row in occupancy:
  print("[", end=" ")
  for item in row:
    if np.isinf(item):
      print(item, ".            ", end=" ")
    else:
      print(item, end=" ")
  print("]")

[ inf .             8.313750693788787 8.742263565356287 4.780942744068623 4.8667877928435495 inf .             inf .             inf .             8.751350418013125 6.235623537529263 ]
[ inf .             8.9987182832472 8.947903521697016 5.832672196572214 9.320621647370995 inf .             inf .             inf .             2.344434449694063 8.102176708796767 ]
[ inf .             3.0964190111768373 8.558410699698815 7.827183051862975 0.0707851490627387 inf .             inf .             inf .             3.016370571063589 2.2380223217102158 ]
[ inf .             9.371183930679907 8.51445823605031 2.888775089148359 8.075096651533602 0.43207099460453313 0.1820845420984596 9.308617489768071 6.290875317984337 0.8116594060623694 ]
[ inf .             6.844854632043951 1.576329843499521 2.5179268853776495 3.6235899577284236 4.597047174505199 3.2358579079526995 8.274915933969524 6.258723248342415 0.14996093364032403 ]
[ inf .             inf .             inf .             inf .         

Espera-se que o mesmo código de cima funcione com esse cenário.

In [ ]:
robot_pos_c = [9, 0] # Robot current position
robot_pos_d = [0, 2] # Robot desired position

path, distance = robot_path(robot_pos_c, robot_pos_d, occupancy)
printPath(path, distance, occupancy)

Starting at (9, 0)
_________________________________________________
From (9, 0) to (9, 1) with cost 6.222268114601915
_________________________________________________
From (9, 1) to (9, 2) with cost 7.005242716378718
_________________________________________________
From (9, 2) to (9, 3) with cost 8.272623493268087
_________________________________________________
From (9, 3) to (9, 4) with cost 5.981614727500043
_________________________________________________
From (9, 4) to (8, 4) with cost 2.119887369440648
_________________________________________________
From (8, 4) to (8, 5) with cost 9.826691272212292
_________________________________________________
From (8, 5) to (8, 6) with cost 5.887357515683879
_________________________________________________
From (8, 6) to (7, 6) with cost 8.083588538425323
_________________________________________________
From (7, 6) to (6, 6) with cost 8.76324325701264
_________________________________________________
From (6, 6) to (5, 6) with cost 